# 猫狗大战 毕业项目——Fine-tuning DenseNet169

## 开始
导入一切并我们设置所使用的GPU。
- dev0: GTX1070Ti

In [1]:
%matplotlib inline
#import utilities
import os
import shutil
import numpy as np
import random
from tqdm import tqdm  
from time import time
from PIL import Image
import h5py
import pandas as pd
from helper import *

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *
from keras import backend as K

#如果系统上有多块GPU，“0”可以替换成其它GPU的编号
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/pengjun/.conda/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 数据文件处理
训练数据包括12500张猫的图片和12500张狗的图片。我们为数据文件建立symbol link并划分为训练集和验证集，所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

In [2]:
#为数据连理symbol-link
train_data_dir, valid_data_dir, test_data_dir = prepare_data_file()

100%|██████████| 12500/12500 [00:00<00:00, 183990.40it/s]


## 基准模型
作为迁移学习的基础，这里我们使用DenseNet169为基准模型：
- [DenseNet](https://arxiv.org/abs/1608.06993)

在导出预训练模型特征时，我们所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

我们首先冻结所有DenseNet的权重参数，只训练全链接层。我们在融合模型中已经导出了所有训练数据和测试数据在DenseNet上的特征，基于这些特征，我们训练猫狗问题的网络。

In [3]:
#导入训练数据和测试数据
X_train, Y_train, X_test = load_feature_data("feature_densenet169.h5")

In [4]:
#构造模型并显示所有网络层的名称
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [5]:
#训练模型并导出权重参数
filepath="densenet169-tune0-best_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 1s 33us/step - loss: 0.1397 - acc: 0.9492 - val_loss: 0.0317 - val_acc: 0.9914

Epoch 00001: val_loss improved from inf to 0.03170, saving model to densenet169-tune0-best_weight.h5
Epoch 2/20
20000/20000 [==============================] - 0s 12us/step - loss: 0.0375 - acc: 0.9870 - val_loss: 0.0219 - val_acc: 0.9924

Epoch 00002: val_loss improved from 0.03170 to 0.02186, saving model to densenet169-tune0-best_weight.h5
Epoch 3/20
20000/20000 [==============================] - 0s 15us/step - loss: 0.0324 - acc: 0.9884 - val_loss: 0.0198 - val_acc: 0.9928

Epoch 00003: val_loss improved from 0.02186 to 0.01978, saving model to densenet169-tune0-best_weight.h5
Epoch 4/20
20000/20000 [==============================] - 0s 15us/step - loss: 0.0302 - acc: 0.9896 - val_loss: 0.0180 - val_acc: 0.9936

Epoch 00004: val_loss improved from 0.01978 to 0.01799, saving model to densenet169-tune

In [6]:
#在测试集上进行预测并导出预测值
predict_on_model(test_data_dir, X_test, model, "pred-densenet169-tune0.csv")

12500/12500 [==============================] - 0s 27us/step
Found 12500 images belonging to 1 classes.


/home/pengjun/DLND/Cat_vs_Dog/helper.py:130: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(index-1, 'label', y_test[i])


> 这一模型在Kaggle上的得分为0.04590。

## Fine Tuning
我们将放开Densenet中的一些单元的权值，让它们是可学习的，以此训练我们的猫狗分类网络。

### Fine Tune freeze
这里，我们首先在冻结全部权重情况下对全链接层进行训练。我们引入数据增强以获得更为泛化的数据集。

In [7]:
#构造模型
x_input = Input((224, 224, 3))
x_input = Lambda(densenet.preprocess_input)(x_input)

base_model = DenseNet169(input_tensor=x_input, weights='imagenet', include_top=False, pooling = 'avg')

for layer in base_model.layers:
    layer.trainable = False
    
x = Dropout(0.5)(base_model.output)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

gen = ImageDataGenerator(rotation_range=90,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)
val_gen = ImageDataGenerator()
train_generator = gen.flow_from_directory(train_data_dir, (224, 224),shuffle=True, 
                                          batch_size=64,class_mode='binary')
valid_generator = val_gen.flow_from_directory(valid_data_dir, (224, 224), shuffle=True, 
                                          batch_size=32,class_mode='binary')
for i in range(len(model.layers)):
    print(i,model.layers[i].name)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
0 input_2
1 lambda_1
2 zero_padding2d_1
3 conv1/conv
4 conv1/bn
5 conv1/relu
6 zero_padding2d_2
7 pool1
8 conv2_block1_0_bn
9 conv2_block1_0_relu
10 conv2_block1_1_conv
11 conv2_block1_1_bn
12 conv2_block1_1_relu
13 conv2_block1_2_conv
14 conv2_block1_concat
15 conv2_block2_0_bn
16 conv2_block2_0_relu
17 conv2_block2_1_conv
18 conv2_block2_1_bn
19 conv2_block2_1_relu
20 conv2_block2_2_conv
21 conv2_block2_concat
22 conv2_block3_0_bn
23 conv2_block3_0_relu
24 conv2_block3_1_conv
25 conv2_block3_1_bn
26 conv2_block3_1_relu
27 conv2_block3_2_conv
28 conv2_block3_concat
29 conv2_block4_0_bn
30 conv2_block4_0_relu
31 conv2_block4_1_conv
32 conv2_block4_1_bn
33 conv2_block4_1_relu
34 conv2_block4_2_conv
35 conv2_block4_concat
36 conv2_block5_0_bn
37 conv2_block5_0_relu
38 conv2_block5_1_conv
39 conv2_block5_1_bn
40 conv2_block5_1_relu
41 conv2_block5_2_conv
42 conv2_block5_concat
43 conv2_block6_0_bn
44 conv

428 conv5_block9_1_conv
429 conv5_block9_1_bn
430 conv5_block9_1_relu
431 conv5_block9_2_conv
432 conv5_block9_concat
433 conv5_block10_0_bn
434 conv5_block10_0_relu
435 conv5_block10_1_conv
436 conv5_block10_1_bn
437 conv5_block10_1_relu
438 conv5_block10_2_conv
439 conv5_block10_concat
440 conv5_block11_0_bn
441 conv5_block11_0_relu
442 conv5_block11_1_conv
443 conv5_block11_1_bn
444 conv5_block11_1_relu
445 conv5_block11_2_conv
446 conv5_block11_concat
447 conv5_block12_0_bn
448 conv5_block12_0_relu
449 conv5_block12_1_conv
450 conv5_block12_1_bn
451 conv5_block12_1_relu
452 conv5_block12_2_conv
453 conv5_block12_concat
454 conv5_block13_0_bn
455 conv5_block13_0_relu
456 conv5_block13_1_conv
457 conv5_block13_1_bn
458 conv5_block13_1_relu
459 conv5_block13_2_conv
460 conv5_block13_concat
461 conv5_block14_0_bn
462 conv5_block14_0_relu
463 conv5_block14_1_conv
464 conv5_block14_1_bn
465 conv5_block14_1_relu
466 conv5_block14_2_conv
467 conv5_block14_concat
468 conv5_block15_0_bn
469 

In [8]:
#训练模型并保存在验证集上损失函数最小的权重
filepath="densenet169-best_weight_freeze.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=1250,
        epochs=10,
        validation_data=valid_generator,
        validation_steps=150,
        callbacks = callbacks_list)

Epoch 1/10
1250/1250 [==============================] - 604s 483ms/step - loss: 0.1579 - acc: 0.9347 - val_loss: 0.0995 - val_acc: 0.9654

Epoch 00001: val_loss improved from inf to 0.09952, saving model to densenet169-best_weight_freeze.h5
Epoch 2/10
1250/1250 [==============================] - 603s 483ms/step - loss: 0.1374 - acc: 0.9450 - val_loss: 0.0770 - val_acc: 0.9738

Epoch 00002: val_loss improved from 0.09952 to 0.07704, saving model to densenet169-best_weight_freeze.h5
Epoch 3/10
1250/1250 [==============================] - 602s 482ms/step - loss: 0.1340 - acc: 0.9459 - val_loss: 0.0773 - val_acc: 0.9727

Epoch 00003: val_loss did not improve
Epoch 4/10
1250/1250 [==============================] - 600s 480ms/step - loss: 0.1348 - acc: 0.9467 - val_loss: 0.1010 - val_acc: 0.9640

Epoch 00004: val_loss did not improve
Epoch 5/10
1250/1250 [==============================] - 599s 479ms/step - loss: 0.1342 - acc: 0.9463 - val_loss: 0.0751 - val_acc: 0.9727

Epoch 00005: val_loss

In [9]:
predict_on_densenet(12500, 224, 224, test_data_dir, model, "densenet169-best_weight_freeze.h5", "pred-densenet169-freeze.csv")

100%|██████████| 12500/12500 [00:38<00:00, 320.71it/s]


12500/12500 [==============================] - 93s 7ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:183: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 241320.46it/s]


> 这个模型在kaggle的得分是0.08211。

### Fine-tuning 1
对于densenet模型，我们值tuning一个denseblock。

In [10]:
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[370:]:
    layer.trainable = True
    
model.load_weights('densenet169-best_weight_freeze.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
#训练模型并保存在验证集上损失函数最小的权重
filepath="densenet169-best_weight_fine_tuning-1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=1250,
        epochs=10,
        validation_data=valid_generator,
        validation_steps=300,
        callbacks = callbacks_list)

Epoch 1/10
1250/1250 [==============================] - 655s 524ms/step - loss: 0.0776 - acc: 0.9708 - val_loss: 0.0254 - val_acc: 0.9923

Epoch 00001: val_loss improved from inf to 0.02539, saving model to densenet169-best_weight_fine_tuning-1.h5
Epoch 2/10
1250/1250 [==============================] - 656s 525ms/step - loss: 0.0459 - acc: 0.9827 - val_loss: 0.0602 - val_acc: 0.9837

Epoch 00002: val_loss did not improve
Epoch 3/10
1250/1250 [==============================] - 655s 524ms/step - loss: 0.0359 - acc: 0.9865 - val_loss: 0.0215 - val_acc: 0.9934

Epoch 00003: val_loss improved from 0.02539 to 0.02154, saving model to densenet169-best_weight_fine_tuning-1.h5
Epoch 4/10
1250/1250 [==============================] - 656s 525ms/step - loss: 0.0277 - acc: 0.9898 - val_loss: 0.0352 - val_acc: 0.9901

Epoch 00004: val_loss did not improve
Epoch 5/10
1250/1250 [==============================] - 656s 525ms/step - loss: 0.0235 - acc: 0.9913 - val_loss: 0.0304 - val_acc: 0.9919

Epoch 0

In [12]:
predict_on_densenet(12500, 224, 224, test_data_dir, model, "densenet169-best_weight_fine_tuning-1.h5", "pred-densenet169-fine_tuning-1.csv")

100%|██████████| 12500/12500 [00:39<00:00, 319.15it/s]


12500/12500 [==============================] - 94s 7ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:183: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 248676.90it/s]


> 这一轮Tuning的最终得分为：0.04314。

In [13]:
base_model.save_weights("fine_tuned_densenet169.h5")